In [ ]:
# -*- coding: utf-8 -*-
"""
Análisis Exploratorio de Datos (EDA) y Optimización de Hiperparámetros
de un modelo LSTM con PyTorch y Optuna para la predicción de Casos de Dengue.

Este script realiza lo siguiente:
1. Carga y prepara los datos de entrenamiento desde un archivo Parquet.
2. Realiza un EDA detallado de la serie de tiempo (estacionariedad, descomposición, autocorrelación).
3. Implementa una red LSTM usando PyTorch.
4. Utiliza Optuna para encontrar los mejores hiperparámetros para el modelo LSTM.
5. Entrena el modelo final con los mejores hiperparámetros y visualiza los resultados.
6. Utiliza el modelo final para predecir los casos de dengue para 2022.
7. Genera un archivo de submission con el formato requerido.
"""

# 1. Importación de Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Librerías para el EDA de series de tiempo
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import optuna

# Configuración de estilo para las gráficas
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# --- ANÁLISIS DEL CONJUNTO DE ENTRENAMIENTO (df_train.parquet) ---

# 2. Carga y Preparación de Datos
print("Cargando datos de entrenamiento...")
DATA_DIR = '../../Datos/'
try:
    # Cargar desde el formato Parquet en el directorio especificado
    df_train = pd.read_parquet(os.path.join(DATA_DIR, 'df_train.parquet'))
    df_train['date'] = pd.to_datetime(df_train['anio'].astype(str) + '-' + df_train['semana'].astype(str) + '-1', format='%Y-%W-%w')
    df_train.set_index('date', inplace=True)
except FileNotFoundError:
    print(f"Error: El archivo 'df_train.parquet' no se encontró en el directorio '{DATA_DIR}'. Asegúrate de que la ruta es correcta.")
    exit()

# 3. Agregación y Visualización de la Serie de Tiempo
dengue_series = df_train['dengue'].resample('W').sum()
print("\nVisualizando la serie de tiempo de casos de dengue...")
dengue_series.plot(title='Casos de Dengue Agregados por Semana')
plt.ylabel('Número de Casos de Dengue')
plt.xlabel('Fecha')
plt.grid(True)
plt.show()

# --- ANÁLISIS EXPLORATORIO DE DATOS (EDA) DE LA SERIE DE TIEMPO ---

# 4. Análisis de Estacionariedad
print("\n--- Analizando la estacionariedad de la serie ---")
# ... (código del EDA se mantiene igual) ...
# Prueba de Dickey-Fuller Aumentada (ADF)
print('\nResultados de la Prueba de Dickey-Fuller Aumentada:')
dftest = adfuller(dengue_series.dropna(), autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)' % key] = value
print(dfoutput)
if dfoutput['p-value'] < 0.05:
    print("\nInterpretación: La serie es estacionaria.")
else:
    print("\nInterpretación: La serie no es estacionaria.")

# 5. Descomposición de la Serie de Tiempo
print("\n--- Descomponiendo la serie de tiempo... ---")
if len(dengue_series) >= 104:
    decomposition = seasonal_decompose(dengue_series, model='additive', period=52)
    fig = decomposition.plot()
    fig.set_size_inches(12, 10)
    plt.show()

# 6. Gráficas de Autocorrelación (ACF) y Autocorrelación Parcial (PACF)
print("\n--- Generando gráficas de autocorrelación (ACF) y PACF... ---")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
plot_acf(dengue_series.dropna(), ax=ax1, lags=52)
ax1.set_title('Función de Autocorrelación (ACF)')
plot_pacf(dengue_series.dropna(), ax=ax2, lags=52)
ax2.set_title('Función de Autocorrelación Parcial (PACF)')
plt.show()


# --- IMPLEMENTACIÓN DE LSTM CON PYTORCH Y OPTUNA ---

print("\n--- Iniciando optimización de hiperparámetros para LSTM con PyTorch y Optuna ---")

# 7. Preparación de Datos para LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dengue_series.values.reshape(-1, 1))

def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

# 8. Definición del Modelo LSTM en PyTorch
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, dropout_rate, output_size=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq)
        last_output = lstm_out[:, -1, :]
        predictions = self.linear(self.dropout(last_output))
        return predictions

# 9. Definición de la Función Objetivo de Optuna
def objective(trial):
    time_step = trial.suggest_int('time_step', 10, 60)
    hidden_layer_size = trial.suggest_int('hidden_layer_size', 32, 200, step=16)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop'])
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    X, y = create_dataset(scaled_data, time_step)
    if len(X) < 2: return float('inf')
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    X_train_tensor = torch.from_numpy(X_train).float().view(-1, time_step, 1)
    y_train_tensor = torch.from_numpy(y_train).float().view(-1, 1)
    X_val_tensor = torch.from_numpy(X_val).float().view(-1, time_step, 1)
    y_val_tensor = torch.from_numpy(y_val).float().view(-1, 1)

    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)

    model = LSTMModel(input_size=1, hidden_layer_size=hidden_layer_size, dropout_rate=dropout_rate)
    loss_function = nn.MSELoss()
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=learning_rate)
    
    epochs, patience, best_val_loss, epochs_no_improve = 100, 10, float('inf'), 0
    
    for epoch in range(epochs):
        model.train()
        for seq, labels in train_loader:
            optimizer.zero_grad()
            y_pred = model(seq)
            single_loss = loss_function(y_pred, labels)
            single_loss.backward()
            optimizer.step()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for seq, labels in val_loader:
                val_loss += loss_function(model(seq), labels).item()
        val_loss /= len(val_loader)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_no_improve += 1
        if epochs_no_improve == patience: break

    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    all_preds = [model(seq).detach().numpy() for seq, _ in val_loader]
    
    predictions = np.concatenate(all_preds)
    predictions_inv = scaler.inverse_transform(predictions)
    y_val_inv = scaler.inverse_transform(y_val.reshape(-1, 1))
    
    return np.sqrt(mean_squared_error(y_val_inv, predictions_inv))

# 10. Ejecución del Estudio de Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, timeout=600)

# 11. Resultados y Entrenamiento del Modelo Final
best_params = study.best_params
print(f"\nMejor trial: RMSE = {study.best_value:.4f}")
print("Mejores hiperparámetros:", best_params)

print("\n--- Entrenando el modelo final con los mejores hiperparámetros... ---")
time_step = best_params['time_step']
X, y = create_dataset(scaled_data, time_step)
X_tensor = torch.from_numpy(X).float().view(-1, time_step, 1)
y_tensor = torch.from_numpy(y).float().view(-1, 1)
final_loader = DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=best_params['batch_size'], shuffle=True)

final_model = LSTMModel(1, best_params['hidden_layer_size'], best_params['dropout_rate'])
loss_function = nn.MSELoss()
optimizer = getattr(optim, best_params['optimizer'])(final_model.parameters(), lr=best_params['learning_rate'])

for i in range(150):
    for seq, labels in final_loader:
        optimizer.zero_grad()
        final_model.train()
        y_pred = final_model(seq)
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
    if (i+1)%25 == 0: print(f'Epoch: {i+1:3} loss: {single_loss.item():.8f}')

# --- PREDICCIÓN PARA 2022 Y GENERACIÓN DE SUBMISSION ---

print("\n--- Prediciendo para 2022 y generando archivo de submission... ---")

# 12. Predicción iterativa para las 52 semanas de 2022
n_weeks_2022 = 52
# Usar los últimos 'time_step' días del training data como secuencia inicial
test_input = scaled_data[-time_step:].flatten().tolist()
predictions_2022_scaled = []

final_model.eval()
with torch.no_grad():
    for _ in range(n_weeks_2022):
        seq = torch.FloatTensor(test_input[-time_step:]).view(1, time_step, 1)
        next_pred = final_model(seq).item()
        predictions_2022_scaled.append(next_pred)
        test_input.append(next_pred)

# 13. Procesar las predicciones
# Desescalar las predicciones
predictions_2022_total = scaler.inverse_transform(np.array(predictions_2022_scaled).reshape(-1, 1))
# Asegurar que los casos sean enteros no negativos
predictions_2022_total = np.maximum(0, predictions_2022_total.round()).astype(int).flatten()

# Calcular el número de barrios para distribuir la predicción
num_neighborhoods = df_train['id_bar'].nunique()
# Distribuir el total predicho equitativamente entre los barrios
predictions_per_neighborhood = predictions_2022_total / num_neighborhoods
# Redondear a entero
predictions_per_neighborhood = predictions_per_neighborhood.round().astype(int)

# 14. Crear el archivo de submission
try:
    df_test = pd.read_parquet(os.path.join(DATA_DIR, 'df_test.parquet'))
    # Crear un mapa de semana -> predicción por barrio
    week_to_prediction_map = {i + 1: pred for i, pred in enumerate(predictions_per_neighborhood)}
    
    # Asignar las predicciones al dataframe de test
    df_test['dengue'] = df_test['semana'].map(week_to_prediction_map)
    
    # Crear el dataframe final para el submission
    submission_df = df_test[['id', 'dengue']].copy()
    
    # Guardar el archivo
    submission_df.to_csv('submission.csv', index=False)
    
    print("\nArchivo 'submission.csv' generado exitosamente.")
    print(submission_df.head())

except FileNotFoundError:
    print(f"Error: El archivo 'df_test.parquet' no se encontró en el directorio '{DATA_DIR}'. No se pudo generar el archivo de submission.")

print("\nProceso completado.")

